In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openpyxl as xl
from openpyxl.chart import BarChart, Reference
import xlrd
import requests
from bs4 import BeautifulSoup

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from operator import indexOf
from bs4 import BeautifulSoup


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# get the url`s
loc = ("/content/drive/MyDrive/code/20211030 Test Assignment/Input.xlsx")
# to open workbook
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
# For row 0 and column 0

def par(url,i):
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',}
  r = requests.get(url, headers=headers)
  htmlContent = r.content
  soup = BeautifulSoup(htmlContent, 'html.parser')
  paras = soup.find_all('p')
  reviews = [result.text for result in paras]
  # for i in paras:
  #   # print(len(i.text))
  #   # print(para.text.encode("utf-8"))
  #   print(i.text)
    

In [ ]:
# get the positive words and negative words
pathn='/content/drive/MyDrive/code/20211030 Test Assignment/MasterDictionary/negative-words.txt'
pathp='/content/drive/MyDrive/code/20211030 Test Assignment/MasterDictionary/positive-words.txt'
pos=['a+']
neg=['2-faced']
poss=0
negg=0
textp= pd.read_csv(pathp, sep='|', encoding='latin-1')
for i in textp['a+']:
  pos.append(i.strip())

textn= pd.read_csv(pathn, sep='|', encoding='latin-1')
for n in textn['2-faced']:
  neg.append(n.strip())



In [ ]:
# get string of stop words
# list all files in the directory

path='/content/drive/MyDrive/code/20211030 Test Assignment/StopWords/'
files = os.listdir(path)
# print(files)
stoptext1=[]

for i in files:
    f = open(path+i, 'r')
    text= pd.read_csv(path+i, sep='|', encoding='latin-1')
    x=[line.strip() for line in text]
    stoptext1.append(x)
    f.close()

print(stoptext1)


[['SMITH', 'Surnames from 1990 census > .002%.  www.census.gov.genealogy/names/dist.all.last'], ['UNITED', 'Geographic'], ['a'], ['HUNDRED', 'Denominations'], ['AFGHANI', 'Afghanistan'], ['ERNST'], ['ABOUT']]


In [ ]:
exlm=['?','!','.','/','-','`','(',')','...','..','....','@','_']

In [ ]:
# function to remove stop words
def rem_stop_word(tokens):
  for i in tokens:
    for j in stoptext1:
      for k in j:
        if(i==k):
          tokens.remove(i)
    for ex in exlm:
        if(i==ex):
          tokens.remove(i)

In [ ]:
# a function to get sillibals
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

# print(syllable_count('chocolate'))

In [ ]:
data = {'URL':['test'],
        'Polarity Score':[0],
        'Subjectivity Score':[0],
        'Average Sentence Length':[0],
        '% of Complex words':[0],
        'Fog Index':[0],
        'Average Words/Sentence':[0]}

df = pd.DataFrame(data)

In [ ]:
# text analysis

def score(stri,url):
  poss=0
  negg=0
  cmplex=0
  se=[]
  seew=[]
  se1=[]
  results=[]
  sent= nltk.sent_tokenize(stri)
  for sen in sent:
    se.append(nltk.sent_tokenize(sen))
    seew=word_tokenize(stri)
  rem_stop_word(seew)

  tokens = nltk.word_tokenize(stri)
  rem_stop_word(tokens)

  for i in tokens:
    for j in pos:
      if(i==j):
        poss=+1
        # print(j)

  for i in tokens:
    for j in neg:
      if(i==j):
        negg=+1
        # print(j)

  # Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
  pscore=(poss-negg)/((poss+negg)+0.000001)
  print('Polarity Score =',pscore)
  # Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
  sscore=(poss+negg)/((len(tokens))+0.000001)
  print('Subjectivity Score = ',sscore)
  # Average Sentence Length = the number of words / the number of sentences
  slen=len(tokens)/len(sent)
  print('Average Sentence Length =',slen)
  # Percentage of Complex words = the number of complex words / the number of words 
  for i in tokens:
    x=syllable_count(i)
    if (x>2):
      cmplex=+1
  comp_word_naalysis=cmplex/len(tokens)
  print('Percentage of Complex words =',comp_word_naalysis)
  # Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
  fog_index= 0.4 * (slen/comp_word_naalysis)
  print('Fog Index =',fog_index)
  # Average Number of Words Per Sentence = the total number of words / the total number of sentences
  for xxe in se:
    x=nltk.word_tokenize(xxe[0])
    se1.append(len(x))
  w_per_sen=np.average(se1)
  print('Average Number of Words Per Sentence =',w_per_sen)

  # df.append({'URL':url,
  #       'Polarity Score':pscore,
  #       'Subjectivity Score':sscore,
  #       'Average Sentence Length':slen,
  #       '% of Complex words':comp_word_naalysis,
  #       'Fog Index':fog_index,
  #       'Average Words/Sentence':w_per_sen},ignore_index=True)

  return ({'URL':url,
        'Polarity Score':pscore,
        'Subjectivity Score':sscore,
        'Average Sentence Length':slen,
        '% of Complex words':comp_word_naalysis,
        'Fog Index':fog_index,
        'Average Words/Sentence':w_per_sen})

In [ ]:
# get the data from the url
def getdata(urll):
  urlstr=''
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
  r = requests.get(urll)
  htmlContent = r.content
  soup = BeautifulSoup(requests.get(urll, headers=headers).content, "html.parser")
  paras = soup.find_all('p')
  for para in paras:
      # print(str(para.text.encode("utf-8")))
      urlstr=urlstr+ str(para.text.encode("utf-8"))
  return(urlstr)

In [ ]:
# for i in range(1,sheet.nrows):
for i in range(1,3):
  print(str(i)+") "+sheet.cell_value(i, 1))
  urlst=getdata(sheet.cell_value(i, 1))
  x=score(urlst,str(sheet.cell_value(i, 1)))
  print(x)
  df.append(x,ignore_index=True)

1) https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
Polarity Score = 0.0
Subjectivity Score =  0.0010576414589859114
Average Sentence Length = 25.904109589041095
Percentage of Complex words = 0.0005288207297726071
Fog Index = 19593.868493150683
Average Number of Words Per Sentence = 27.397260273972602
2) https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
Polarity Score = 0.0
Subjectivity Score =  0.0013351134837549308
Average Sentence Length = 19.710526315789473
Percentage of Complex words = 0.0006675567423230974
Fog Index = 11810.547368421052
Average Number of Words Per Sentence = 21.105263157894736
